In [1]:
import networkx as nx
import numpy as np
import fugu
from fugu import Scaffold, Brick, Spike_Input, Threshold

# Shortest Path Example

This example is not finished yet.  Right now, all we can do is find the length of the shortest path between two nodes.

In [2]:
#Helper function to make a random graph for testing
def create_random_graph(num_nodes, max_weight):
    graph = nx.Graph()
    nodes = list(range(0,num_nodes))
    for node in range(0, num_nodes):
        graph.add_node(node)
    for node in range(0, num_nodes):
        other_nodes = nodes[0:node]+nodes[node+1:num_nodes]
        neighbors = np.random.choice(other_nodes, np.random.randint(1,len(other_nodes)) )
        for neighbor in neighbors:
            graph.add_edge(node, neighbor, weight=np.random.randint(1,max_weight+1))
    return graph


In [3]:
class Shortest_Path_Length(Brick):
    def __init__(self, target_graph, target_node, name=None, output_coding = 'temporal-L'):
        super(Brick, self).__init__()
        #The brick hasn't been built yet.
        self.is_built = False
        #We just store the name passed at construction.
        self.name = name
        #For this example, we'll let any input coding work even though the answer might not make sense.
        self.supported_codings = fugu.input_coding_types
        #Right now, we'll convert node labels to integers in the order of
        #graph.nodes() However, in the fugure, this should be improved to be 
        #more flexible.
        for i,node in enumerate(target_graph.nodes()):
            if node is target_node:
                self.target_node = i
        self.target_graph = nx.relabel.convert_node_labels_to_integers(target_graph)
        self.output_codings = [output_coding]
        self.dimensionality = {'D':None}
    def build(self,
             graph,
             dimensionality,
             control_nodes,
             input_lists,
             input_codings):
        
        if len(input_lists) is not 1:
            raise ValueError('Incorrect Number of Inputs.')
        for input_coding in input_codings:
            if input_coding not in self.supported_codings:
                raise ValueError("Unsupported Input Coding. Found: " + input_coding + ". Allowed: " + str(self.supported_codings))
        
        #All bricks should provide a neuron that spikes when the brick has completed processing.
        #We just put in a basic relay neuron that will spike when it recieves any spike from its
        #single input, which is the complete_node from the first input.
        #All nodes we add to the graph should have basic neuron parameters (threshold, decay)
        #Reasonable defaults will be filled-in, but these defaults may depend on the execution platform.
        #Additionally, nodes should have a field called 'index' which is a local index used to reference the 
        #position of the node.  This can be used by downstream bricks.  A simple example might be
        #a 3-bit binary representation will add 3 nodes to the graph with indices 0,1,2
        #We do have to do some work to establish best practices here.
        #new_complete_node_name = self.name + '_complete'
        #graph.add_node(new_complete_node_name,
        #              index = -1,
        #              threshold = 0.0,
        #              decay =0.0,
        #              p=1.0,
        #              potential=0.0)
        #complete_node = [new_complete_node_name]
        new_begin_node_name = self.name+'_begin'
        graph.add_node(new_begin_node_name,
                      threshold = 0.5,
                      decay = 0.0,
                      potential=0.0)
        graph.add_edge(control_nodes[0]['complete'],
                      self.name+'_begin',
                      weight = 1.0,
                      delay = 1)
        
        
        for node in self.target_graph.nodes:
            graph.add_node(self.name+str(node),
                           index = (node,),
                           threshold=1.0, 
                           decay=0.0, 
                           potential=0.0)
            graph.add_edge(self.name+str(node), self.name+str(node), weight=-1000, delay=1)
            if node==self.target_node:
                complete_node_list = [self.name+str(node)]
                #graph.add_edge(self.name+str(node),
                #       new_complete_node_name,
                #       weight=1.0,delay=1)

        for node in self.target_graph.nodes:
            neighbors = list(self.target_graph.neighbors(node))
            for neighbor in neighbors:
                delay = self.target_graph.edges[node,neighbor]['weight']
                graph.add_edge(self.name+str(node), self.name+str(neighbor), weight=1.5, delay=delay)
                #graph.add_edge(neighbor, node, weight=1.5, delay=delay)

        
        for input_neuron in input_lists[0]:
            index = graph.nodes[input_neuron]['index']
            if type(index) is tuple:
                index = index[0]
            if type(index) is not int:
                raise TypeError("Neuron index should be Tuple or Int.")
            graph.add_edge(input_neuron, 
                          self.name+str(index),
                         weight = 2.0,
                         delay = 1)

        self.is_built=True
        
        #Remember, bricks can have more than one output, so we need a list of list of output neurons
        output_lists = [[self.name+str(self.target_node)]]
        
        return (graph,
               self.dimensionality,
                [{'complete':complete_node_list[0], 'begin':new_begin_node_name}],
                output_lists,
                self.output_codings
               )

In [10]:
g = create_random_graph(5,1)

In [11]:
scaffold = Scaffold()
scaffold.add_brick(Spike_Input(np.array([0,1,0,0,0]), coding='Raster', name='Input0'), 'input' )
scaffold.add_brick(Shortest_Path_Length(g,0))
scaffold.add_brick(Threshold(1),output=True)
scaffold.lay_bricks()
result = scaffold.evaluate(backend='ds',record_all=True)

In [12]:
result

{0: [1, 5],
 1: [6, 8],
 2: [7, 9, 11, 12, 13],
 3: [10],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: []}

In [7]:
g.edges

EdgeView([(0, 2), (0, 1), (0, 3), (0, 4), (1, 3), (1, 4), (2, 3), (3, 4)])

In [16]:
assert(type(3) is int)

In [13]:
for i, node in enumerate(scaffold.graph.nodes):
    print(str(i) + ":" + str(scaffold.graph.nodes[node]))

0:{'index': (0,), 'threshold': 0.0, 'decay': 0.0, 'p': 1.0}
1:{'index': (1,), 'threshold': 0.0, 'decay': 0.0, 'p': 1.0}
2:{'index': (2,), 'threshold': 0.0, 'decay': 0.0, 'p': 1.0}
3:{'index': (3,), 'threshold': 0.0, 'decay': 0.0, 'p': 1.0}
4:{'index': (4,), 'threshold': 0.0, 'decay': 0.0, 'p': 1.0}
5:{'index': -1, 'threshold': 0.0, 'decay': 0.0, 'p': 1.0, 'potential': 0.5}
6:{'threshold': 0.5, 'decay': 0.0, 'potential': 0.0}
7:{'index': (0,), 'threshold': 1.0, 'decay': 0.0, 'potential': 0.0}
8:{'index': (1,), 'threshold': 1.0, 'decay': 0.0, 'potential': 0.0}
9:{'index': (2,), 'threshold': 1.0, 'decay': 0.0, 'potential': 0.0}
10:{'index': (3,), 'threshold': 1.0, 'decay': 0.0, 'potential': 0.0}
11:{'index': (4,), 'threshold': 1.0, 'decay': 0.0, 'potential': 0.0}
12:{'index': -1, 'threshold': -9.999999999998899e-05, 'decay': 1.0, 'p': 1.0}
13:{'index': (0,), 'threshold': 1.0, 'decay': 0.0, 'p': 1.0, 'record': ['spikes']}


In [8]:
scaffold.circuit.nodes[1]

{'name': "<class '__main__.Shortest_Path_Length'>_0",
 'brick': <__main__.Shortest_Path_Length at 0xa2125ad30>,
 'input_nodes': [(0, 0)],
 'dimensionality': {'D': None},
 'output_codings': ['temporal-L'],
 'output_lists': [["<class '__main__.Shortest_Path_Length'>_00"]],
 'control_nodes': [{'complete': "<class '__main__.Shortest_Path_Length'>_00",
   'begin': "<class '__main__.Shortest_Path_Length'>_0_begin"}]}

In [176]:
a = [-2, 0, 1,2,3]

for i in [i_n for i_n in a if i_n > 0]:
    print(a)

[-2, 0, 1, 2, 3]
[-2, 0, 1, 2, 3]
[-2, 0, 1, 2, 3]


In [6]:
str((2,1))

'(2, 1)'